In [6]:
def set_remain(PrecursorMz):
    """
    返回列表去重结果 和 剩余结果
    :param PrecursorMz: 原列表 即母离子的mz值列表
    :return: x1为: set集合去重得到的结果 
            x2为: 列表减去-x1 
            idx为: x1所在原列表中的索引下标
    """
    idx = []
    x1 = sorted(set(PrecursorMz), key=PrecursorMz.index)  # 去重原来顺序不变
    for item in x1:
        idx.append(PrecursorMz.index(item))
    dic = {i:element for i, element in enumerate(PrecursorMz)}  # 给元素变为字典 进行索引批量删除
    [dic.pop(i) for i in idx]
    x2 = list(dic.values())
    return (x1,x2,idx)

In [11]:
"""
    此方法的目的是按照 PrecursorMz(为母离子的mz值)的值 进行整理文件
    因为skyline要求每个csv文件里不能有相同的母离子mz值 
"""
import pandas as pd

# 读文件 不包含header
filename = 'pos.csv'
ion = filename.split('.')[0]
input_data = pd.read_csv(filename, encoding='gbk').values  

# header 第一行行名
header = pd.read_csv(filename, encoding='gbk',header=None).values[0,:]

# PrecursorMz为母离子的mz值
PrecursorMz = list(input_data[:,1])  

# 临时变量 copy一份PrecursorMz
tmp = PrecursorMz

# 第一次获得列表的去重结果的索引下标
list_idx = []
set_1 = sorted(set(tmp), key=tmp.index)
for item in set_1:
    list_idx.append(PrecursorMz.index(item))

# 输入的数据转为DataFrame
df = pd.DataFrame(input_data)
print("input_data")
print(df)
print("-"*100)
print('\n')

# 得到x1 即set
set_data = df.iloc[list_idx, :]
print("out_data")
print(set_data)
print("*"*100)

# 写文件
file1_name = ion + '_level_1.csv'
set_data.to_csv(file1_name, index=False, header=header)


# 调用函数
x1, x2, idx = set_remain(PrecursorMz)
# 去掉x1所在的行 得到剩下的行 即remain
df = df.drop(index=idx)

# i用来给文件名计数
i = 2
# 循环 x2为空时循环结束
while len(x2):
    PrecursorMz = x2
    x1, x2, idx = set_remain(PrecursorMz)
    # print("x1, x2, idx----------", x1, x2, idx)

    # 删除之后需要重置索引 使其下标从0开始
    df = df.reset_index(drop=True)

    # 得到x1 即set
    set_data = df.iloc[idx, :]
    print(set_data)
    print("*"*100)

    # 写文件
    write_file_name = ion + "_level_" + str(i) + ".csv"
    set_data.to_csv(write_file_name,index=False,header=header)

    # 去掉x1所在的行 得到剩下的行 即remain
    df = df.drop(index=idx)

    # 文件名计数加1
    i += 1


input_data
                                                     0       1       2  3  4  \
0                                         Pimelic acid  143.07   69.07  1  1   
1                                   X5.9_151.075mz_pos  151.08  123.08  1  1   
2                                         Azelaic acid   171.1   97.01  1  1   
3                                             Cotinine   177.1   80.05  1  1   
4    2-(1-Hydroxyethyl)-4-(2-hydroxypropyl)-2H-fura...   187.1  111.04  1  1   
..                                                 ...     ...     ... .. ..   
139      hexadecanedioic acid mono-L-carnitine ester-3  430.31   85.03  1  1   
140                        Glycoursodeoxycholic acid-3  432.31   76.04  1  1   
141                                X15_504.316mz_pos-3  504.32  257.17  1  1   
142                                     PC(20:1/0:0)-4  550.39  184.07  1  1   
143                                     Azelaic acid-5   171.1   97.01  1  1   

        5     6  
0    2.19 